## 决策树算法实现流程：

* 1、 准备数据，数据须为没有缺失值的向量，可通过数据清洗和数据处理等方式
* 2、 寻找划分属性特征
* 3、 生成分支
* 4、 生成决策树

我们现在要构建一颗树，如何去构建树呢？我们可以回想到之前学习过的ID3算法中利用信息增益来计算如何寻找划分属性，我们可以一直寻找最佳的属性来进行划分

#### 生成数据集

In [1]:
import pandas as pd
data=pd.read_csv('watermelon_3a.csv')
data

,Idx,color,root,knocks,texture,navel,touch,density,sugar_ratio,label
0,1,dark_green,curl_up,little_heavily,distinct,sinking,hard_smooth,0.697,0.460,1
1,2,black,curl_up,heavily,distinct,sinking,hard_smooth,0.774,0.376,1
2,3,black,curl_up,little_heavily,distinct,sinking,hard_smooth,0.634,0.264,1
3,4,dark_green,curl_up,heavily,distinct,sinking,hard_smooth,0.608,0.318,1
4,5,light_white,curl_up,little_heavily,distinct,sinking,hard_smooth,0.556,0.215,1
5,6,dark_green,little_curl_up,little_heavily,distinct,little_sinking,soft_stick,0.403,0.237,1
6,7,black,little_curl_up,little_heavily,little_blur,little_sinking,soft_stick,0.481,0.149,1
7,8,black,little_curl_up,little_heavily,distinct,little_sinking,hard_smooth,0.437,0.211,1
8,9,black,little_curl_up,heavily,little_blur,little_sinking,hard_smooth,0.666,0.091,0
9,10,dark_green,stiff,clear,distinct,even,soft_stick,0.243,0.267,0


#### 信息熵

In [25]:
import numpy as np
def calcShannonEnt(dataSet):
    '''
    dataSet__数据集
    '''
    numEntries = len(data)           # 计算公式中的D
    labelCounts = { }                 # 存储数据的格式
    # 我们先拿到最后一列lable的标签
    for i in range(numEntries):
        each_data = data.iloc[i,:]
        currentLabel = each_data[-1]
        #print(currentLabel)


        # 统计频数
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] +=1
        print(labelCounts)
        print(labelCounts[1])

    # 计算香农熵
    shannonEnt = 0.0 # 初始化
    for key in labelCounts:
        prob = float(labelCounts[key]) / numEntries
        # print(labelCounts[key])
        shannonEnt -= prob * np.log2(prob)
        print(shannonEnt)
    return shannonEnt


$Info(D) = -\sum_{i=1}^{m}P(i)logP(i)$

* 在写code之前，我们得需明白公式中的参数在数据集中代表什么含义
* P(i)___利用频率计算出来的概率值(类别/整个数据集)
* 是要用什么数据集来进行计算___整个数据集
* {"类别": 频率}___可以用字典格式来存储
* 对整个数据集遍历，每次遍历的是一个样本
* 可以通过特征的索引值拿到相应的属性

In [5]:
data

,Idx,color,root,knocks,texture,navel,touch,density,sugar_ratio,label
0,1,dark_green,curl_up,little_heavily,distinct,sinking,hard_smooth,0.697,0.460,1
1,2,black,curl_up,heavily,distinct,sinking,hard_smooth,0.774,0.376,1
2,3,black,curl_up,little_heavily,distinct,sinking,hard_smooth,0.634,0.264,1
3,4,dark_green,curl_up,heavily,distinct,sinking,hard_smooth,0.608,0.318,1
4,5,light_white,curl_up,little_heavily,distinct,sinking,hard_smooth,0.556,0.215,1
5,6,dark_green,little_curl_up,little_heavily,distinct,little_sinking,soft_stick,0.403,0.237,1
6,7,black,little_curl_up,little_heavily,little_blur,little_sinking,soft_stick,0.481,0.149,1
7,8,black,little_curl_up,little_heavily,distinct,little_sinking,hard_smooth,0.437,0.211,1
8,9,black,little_curl_up,heavily,little_blur,little_sinking,hard_smooth,0.666,0.091,0
9,10,dark_green,stiff,clear,distinct,even,soft_stick,0.243,0.267,0


In [10]:
data = data.loc[   :  , 'color':'sugar_ratio']
data

,color,root,knocks,texture,navel,touch,density,sugar_ratio
0,dark_green,curl_up,little_heavily,distinct,sinking,hard_smooth,0.697,0.460
1,black,curl_up,heavily,distinct,sinking,hard_smooth,0.774,0.376
2,black,curl_up,little_heavily,distinct,sinking,hard_smooth,0.634,0.264
3,dark_green,curl_up,heavily,distinct,sinking,hard_smooth,0.608,0.318
4,light_white,curl_up,little_heavily,distinct,sinking,hard_smooth,0.556,0.215
5,dark_green,little_curl_up,little_heavily,distinct,little_sinking,soft_stick,0.403,0.237
6,black,little_curl_up,little_heavily,little_blur,little_sinking,soft_stick,0.481,0.149
7,black,little_curl_up,little_heavily,distinct,little_sinking,hard_smooth,0.437,0.211
8,black,little_curl_up,heavily,little_blur,little_sinking,hard_smooth,0.666,0.091
9,dark_green,stiff,clear,distinct,even,soft_stick,0.243,0.267


In [23]:
# 对离散变量进行划分数据集，
def splitDataSet(dataSet, axis, value):
    '''
    dataSet___数据集
    axis___特征索引
    value__特征值
    
    example: axis = 1，color这个特征，value=black, light_white等
    '''
    reDataSet = []    # 不修改原始数据集，创建一个新的列表对象
    # data原本是DataFrame格式，通过data.values可以将DataFrame格式转换为array
    for featVec in data.values:     #循环遍历每一个样本
        # print(featVec[2])
        if featVec[axis] == value:
            print(featVec[axis])
            reduceFeatVec = featVec[:axis]
            reduceFeatVec1 = list(reduceFeatVec)
            # ndarray没有extend()函数，需要转换为list
            reduceFeatVec1.extend(featVec[axis+1:])   # 跳过axis对应的属性 
            reDataSet.append(reduceFeatVec1)
            
    return reDataSet,type(reDataSet)

In [24]:
# 测试函数
splitDataSet(data, 1, "curl_up")

curl_up
curl_up
curl_up
curl_up
curl_up
curl_up
curl_up
curl_up


([['dark_green',
   'little_heavily',
   'distinct',
   'sinking',
   'hard_smooth',
   0.6970000000000001,
   0.46],
  ['black', 'heavily', 'distinct', 'sinking', 'hard_smooth', 0.774, 0.376],
  ['black',
   'little_heavily',
   'distinct',
   'sinking',
   'hard_smooth',
   0.634,
   0.264],
  ['dark_green',
   'heavily',
   'distinct',
   'sinking',
   'hard_smooth',
   0.608,
   0.318],
  ['light_white',
   'little_heavily',
   'distinct',
   'sinking',
   'hard_smooth',
   0.556,
   0.215],
  ['light_white',
   'little_heavily',
   'blur',
   'even',
   'soft_stick',
   0.34299999999999997,
   0.099],
  ['light_white',
   'little_heavily',
   'blur',
   'even',
   'hard_smooth',
   0.593,
   0.042],
  ['dark_green',
   'heavily',
   'little_blur',
   'little_sinking',
   'hard_smooth',
   0.7190000000000001,
   0.10300000000000001]],
 list)

In [50]:
# 对于我们data数据集来讲，Idx属性下的值都是连续的，所以我们要对其进行离散化
# 连续数据进行离散化，这里我进行二类划分

def splitContinuousDataSet(dataSet, axis, value, direction):
    """
    dataSet___待划分数据
    axis___划分数据集特征的列号，如：0，1...
    value___需要返回特征的值，属性值对应字符串
    """
    reDataSet = []
    for featVec in dataSet.values:
        if direction == 0:
            if featVec[axis] >  value:
                reduceFeatVec = featVec[:axis]
                reduceFeatVecList = list(reduceFeatVec)
                reduceFeatVecList.extend(featVec[axis+1 :])
                reDataSet.append(reduceFeatVec)
        else:
            if featVec[axis] <=  value:
                reduceFeatVec = featVec[:axis]
                reduceFeatVecList = list(reduceFeatVec)
                reduceFeatVecList.extend(featVec[axis+1 :])
                reDataSet.append(reduceFeatVec)
    return reDataSet  

In [51]:
for i in data.values:
    print(i)

['dark_green' 'curl_up' 'little_heavily' 'distinct' 'sinking'
 'hard_smooth' 0.6970000000000001 0.46]
['black' 'curl_up' 'heavily' 'distinct' 'sinking' 'hard_smooth' 0.774
 0.376]
['black' 'curl_up' 'little_heavily' 'distinct' 'sinking' 'hard_smooth'
 0.634 0.264]
['dark_green' 'curl_up' 'heavily' 'distinct' 'sinking' 'hard_smooth' 0.608
 0.318]
['light_white' 'curl_up' 'little_heavily' 'distinct' 'sinking'
 'hard_smooth' 0.556 0.215]
['dark_green' 'little_curl_up' 'little_heavily' 'distinct'
 'little_sinking' 'soft_stick' 0.40299999999999997 0.237]
['black' 'little_curl_up' 'little_heavily' 'little_blur' 'little_sinking'
 'soft_stick' 0.48100000000000004 0.149]
['black' 'little_curl_up' 'little_heavily' 'distinct' 'little_sinking'
 'hard_smooth' 0.43700000000000006 0.21100000000000002]
['black' 'little_curl_up' 'heavily' 'little_blur' 'little_sinking'
 'hard_smooth' 0.6659999999999999 0.091]
['dark_green' 'stiff' 'clear' 'distinct' 'even' 'soft_stick' 0.243 0.267]
['light_white' 'stif

In [52]:
splitContinuousDataSet(data, 7, 0.3, 1)

[array(['black', 'curl_up', 'little_heavily', 'distinct', 'sinking',
        'hard_smooth', 0.634], dtype=object),
 array(['light_white', 'curl_up', 'little_heavily', 'distinct', 'sinking',
        'hard_smooth', 0.556], dtype=object),
 array(['dark_green', 'little_curl_up', 'little_heavily', 'distinct',
        'little_sinking', 'soft_stick', 0.40299999999999997], dtype=object),
 array(['black', 'little_curl_up', 'little_heavily', 'little_blur',
        'little_sinking', 'soft_stick', 0.48100000000000004], dtype=object),
 array(['black', 'little_curl_up', 'little_heavily', 'distinct',
        'little_sinking', 'hard_smooth', 0.43700000000000006], dtype=object),
 array(['black', 'little_curl_up', 'heavily', 'little_blur',
        'little_sinking', 'hard_smooth', 0.6659999999999999], dtype=object),
 array(['dark_green', 'stiff', 'clear', 'distinct', 'even', 'soft_stick',
        0.243], dtype=object),
 array(['light_white', 'stiff', 'clear', 'blur', 'even', 'hard_smooth',
        0.245]

In [31]:
data
featlist = [example[1] for example in data.values]
featlist[0]

'curl_up'

In [36]:
# 选择最好的数据集划分方式
def chooseBestFeatureToSplit(dataSet, labels):
    '''
    dataSet__带划分的数据集
    labels__标签
    '''
    bestFeature = -1  # 初始化
    bestInfoGain = 0.0   # 初始化
    baseEntropy = calcShannonEnt(dataSet)   # 计算数据集总的信息熵
    numFeatures = len(dataSet[0])-1         # 特征个数
    
    # 遍历每一个特征，计算信息增益，信息增益最大的那个。然后根据特征值进行划分数据
    for i in range(numFeatures):   # 循环遍历每一个特征
        # 先将遍历出来的特征值放在list里面
        featlist = [example[i] for example in dataSet.values]  # 将每一个特征的特征值全都取了出来
        # print(featlist)
        
        # 对连续值进行处理
        if type(featlist[0]).__name__ == 'float' or type(featlist[0]).__name__ == 'int':
            # 产生n-1个候选划分点
            # 先进行排序
            sortfeatList = sorted(featlist)  # 先进行排序
            # 然后每两个值相加除以2
            splitList = []
            for j in range (len(sortfeatList)-1):
                splitList.append((sortfeatList[j]+sortfeatList[j+1])/2.0)   # 相加除以2，然后添加到splitList
            
            # 上述已经对连续型数据做了处理，接下来就是进行划分数据
            
            bestSplitEntropy = 10000  # 初始化
            slen = len(splitList)  # 离散化后的数据个数
            
            # 求用第j个候选划分带你划分时，得到的信息熵，并记录最佳划分点
            for j in range(slen):
                value = splitList[j]
                newEntropy = 0.0  #初始化
                subDataSet0 = splitContinuousDataSet(dataSet, i, value, 0)
                subDataSet1 = splitContinuousDataSet(dataSet, i, value, 1)
                
                # 计算条件熵
                prob0 = len(subDataSet0) / len(dataSet)
                newEntropy += prob0*calcShannonEnt(subDataSet0)
                
                prob1 = len(subDataSet1) / len(dataSet)
                newEntropy += prob1 * calcShannonEnt(subDataSet1)
                
                if newEntropy < bestSplitEntropy:
                    bestSplitEntropy = newEntropy
                    bestplit = j
                bestSplitDict = {}
                # 用字典记录当前特征的最佳划分点
                bestSplitDict[labels[i]] = bestSplitDict[bestplit]
                
                #计算IfGain 
                infoGain = baseEntropy - bestSplitEntropy
             

        # 对离散特征进行处理处理
        else:
            uniqueVals = set(featList)   # 去重
            newEntropy = 0.0
            
            #计算条件熵
            # 计算该特征下每种划分的信息熵
            for value in uniqueVals:
                subDataSet = splitDataSet(dataSet, i, value)
                proba = len(subDataSet) / float(len(dataSet))
                # 计算条件熵
                numEntropy += proba * calcShannonEnt(subDataSet)
            # 信息增益
            infoGain = baseEntropy- newEntropy
        if infoGain >bestInfoGain:
            bestInfoGain = infoGain
            bestFeature=i
    return bestFeature   # 返回最好的特征

In [53]:
# 若特征已经划分完，节点下的样本还没有统一取值，则需要进行投票
def majorityCnt(classList):
    classCount = {}
    for vote in classList:
        # 统计频数，对象：频数——字典
        # 
        if vote not in classCount.key():
            classCount[vote] = 0
        classCount[vote] += 1
    return max(classCount)    

In [ ]:
# 递归产生树
'''
思路：【递归】：递归需得有停止条件
    1. 若数据集属于同一类，则返回该类别，划分停止
    2. 若数据集所有特征已经遍历，返回当前计数最多的类别为该结点类别，划分停止
    3. 否则继续分支，调用chooseFeature()函数，选择当前数据集最优特征
    4. 遍历当前最优特征各属性值，划分数据集，并递归调用自身createTree()构建子数据集的决策树
'''
def creatTree(dataSet, labels):
    '''
    dataSet__代划分的数据集
    labels__数据集中所有特征的标签
    '''
    # 停止划分条件之一：标签完成相同，则直接返回该类标签
    classList = [example[-1] for example in dataSet]
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    # 停止划分条件之二：使用完了所有特征，仍然不能将数据划分成仅包含唯一类别的分组
    # 则使用majorityCnt函数挑选次数出现最多的类别作为返回值
    if len(dataSet.values[0]) ==1:
        return majorityCnt(classList)  # 挑选次数出现最多的类别作为返回值
    bestFeat = chooseBestFeatureToSplit(dataSet, labels)    # 选择最优特征
    bestFeatLabel = labels[bestFeat]    # 得到最好的特征标签
    myTree = {bestFeatLabel:{}}  # 创建树，储存数的信息
    

    del[labels[bestFeatLabel]]  # 删除已经使用的特征标签
    
    for value in uniqueValsFull:  
        myTree[bestFeatLabel][value]=createTree(splitDataSet(dataSet,bestFeat,value),subLabels)
           
    return myTree         
    
    